In [1]:
# DenseNet - OCTDL - 5CV
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Disable GPU if needed
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Enable Mixed Precision for Faster Training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Paths
dataset_base_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/dataset split/OCTDL_Splits"
logs_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/denseNet/octdl/training"
models_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/denseNet/octdl/models"
results_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/denseNet/octdl/results"

# Hyperparameters
image_size = 224
batch_size = 32
epochs = 50
num_classes = 7

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    zoom_range=0.3,
    shear_range=0.2
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Prepare results storage
all_results = []

for i in range(1, 6):
    print(f"\n========== Training on Fold {i} ==========")

    set_path = os.path.join(dataset_base_path, f"OCTDL_Set_{i}")
    train_dir, val_dir, test_dir = [os.path.join(set_path, x) for x in ["Train", "Validation", "Test"]]

    # Load datasets
    train_data = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical')
    val_data = val_test_datagen.flow_from_directory(val_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical')
    test_data = val_test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', shuffle=False)

    # Compute Class Weights
    labels = train_data.classes
    unique_classes = np.unique(labels)
    class_weights = compute_class_weight('balanced', classes=unique_classes, y=labels)
    class_weight_dict = {i: weight for i, weight in zip(unique_classes, class_weights)}

    # Load Pretrained DenseNet
    base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all but top 50 layers
    for layer in base_model.layers[:-50]:
        layer.trainable = False
    for layer in base_model.layers[-15:]:
        layer.trainable = True

    # Build model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", dtype='float32')  # Match precision policy
    ])

    # Compile model with label smoothing
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=CategoricalCrossentropy(label_smoothing=0.1),
        metrics=["accuracy"]
    )

    # Callbacks
    model_save_path = os.path.join(models_path, f"denseNet_OCTDL_best_fold_{i}.keras")
    callbacks = [
        ModelCheckpoint(model_save_path, monitor='val_accuracy', save_best_only=True, verbose=1),
        EarlyStopping(monitor='val_accuracy', patience=8, verbose=1, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=4, verbose=1),
        CSVLogger(os.path.join(logs_path, f"denseNet_OCTDL_fold_{i}.csv"), append=True)
    ]

    # Train
    model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        class_weight=class_weight_dict,
        callbacks=callbacks
    )

    # Evaluate on Test Set
    test_loss, test_accuracy = model.evaluate(test_data, verbose=1)

    # Predictions
    predictions = model.predict(test_data)
    predicted_classes = np.argmax(predictions, axis=1)
    true_labels = test_data.classes

    # Metrics
    cm = confusion_matrix(true_labels, predicted_classes)
    report = classification_report(true_labels, predicted_classes, target_names=train_data.class_indices.keys(), output_dict=True)

    # Store results
    all_results.append({
        "Fold": i,
        "Test Accuracy": test_accuracy,
        "Test Loss": test_loss,
        "Precision": report["weighted avg"]["precision"],
        "Recall": report["weighted avg"]["recall"],
        "F1-Score": report["weighted avg"]["f1-score"]
    })

# Save results
results_df = pd.DataFrame(all_results)
results_excel_file = os.path.join(results_path, "denseNet_OCTDL_5CV_results_finetuned.xlsx")
results_df.to_excel(results_excel_file, index=False)
print(f"✅ All results saved to {results_excel_file}")



========== Training on Fold 1 ==========
Found 162 images belonging to 7 classes.
Found 65 images belonging to 7 classes.
Found 97 images belonging to 7 classes.


2025-04-02 23:51:24.858443: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-02 23:51:24.858482: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-02 23:51:24.858487: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-04-02 23:51:24.859162: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-02 23:51:24.859184: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super()

Epoch 1/50


2025-04-02 23:51:29.920989: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1009 - loss: 2.8390
Epoch 1: val_accuracy improved from -inf to 0.06154, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/denseNet/octdl/models/denseNet_OCTDL_best_fold_1.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 38s 4s/step - accuracy: 0.1076 - loss: 2.9002 - val_accuracy: 0.0615 - val_loss: 2.6524 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 860ms/step - accuracy: 0.2026 - loss: 3.4192
Epoch 2: val_accuracy improved from 0.06154 to 0.07692, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/denseNet/octdl/models/denseNet_OCTDL_best_fold_1.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.2063 - loss: 3.3537 - val_accuracy: 0.0769 - val_loss: 2.8218 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 839ms/step - accuracy: 0.2942 - loss: 2.0732
Epoch 3: val_accuracy improved from 0

...

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.1612 - loss: 2.9944 
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/denseNet/octdl/models/denseNet_OCTDL_best_fold_4.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 126s 13s/step - accuracy: 0.1648 - loss: 2.9763 - val_accuracy: 0.2000 - val_loss: 1.9221 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 40s/step - accuracy: 0.3059 - loss: 2.5173 
Epoch 2: val_accuracy improved from 0.20000 to 0.23077, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/denseNet/octdl/models/denseNet_OCTDL_best_fold_4.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 223s 44s/step - accuracy: 0.3031 - loss: 2.4893 - val_accuracy: 0.2308 - val_loss: 1.7994 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3038 - loss: 2.2451
Epoch 3: val_accuracy imp

/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.1527 - loss: 2.7580
Epoch 1: val_accuracy improved from -inf to 0.04615, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/denseNet/octdl/models/denseNet_OCTDL_best_fold_5.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 121s 13s/step - accuracy: 0.1548 - loss: 2.7241 - val_accuracy: 0.0462 - val_loss: 3.6471 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1835 - loss: 2.4564
Epoch 2: val_accuracy did not improve from 0.04615
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.1910 - loss: 2.4773 - val_accuracy: 0.0462 - val_loss: 3.1821 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 852ms/step - accuracy: 0.2746 - loss: 2.4417
Epoch 3: val_accuracy did not improve from 0.04615
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.2736 - loss: 2.4344 - val_accuracy: 0.0462 - val_loss: 2.7463 - learning_rate: 1.0000e-04
Epoch 4/

/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

✅ All results saved to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/denseNet/octdl/results/denseNet_OCTDL_5CV_results_finetuned.xlsx


In [3]:
#MobileNet - OCTDL - 5CV
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Enable mixed precision (works well on M1/M2 chips)
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Paths
dataset_base_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/dataset split/OCTDL_Splits"
logs_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/mobilenet/octdl/training"
models_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/mobilenet/octdl/models"
results_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/mobilenet/octdl/results"

# Ensure all log directories exist
os.makedirs(logs_path, exist_ok=True)
os.makedirs(models_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)

# Hyperparameters
image_size = 224
batch_size = 32
epochs = 50
num_classes = 7

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    zoom_range=0.3,
    shear_range=0.2
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Results storage
all_results = []

for i in range(1, 6):
    print(f"\n========== Training on Fold {i} ==========")

    set_path = os.path.join(dataset_base_path, f"OCTDL_Set_{i}")
    train_dir, val_dir, test_dir = [os.path.join(set_path, x) for x in ["Train", "Validation", "Test"]]

    # Load data
    train_data = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical')
    val_data = val_test_datagen.flow_from_directory(val_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical')
    test_data = val_test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', shuffle=False)

    # Compute class weights
    labels = train_data.classes
    unique_classes = np.unique(labels)
    class_weights = compute_class_weight('balanced', classes=unique_classes, y=labels)
    class_weight_dict = {i: weight for i, weight in zip(unique_classes, class_weights)}

    # Load MobileNetV2
    base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))
    
    # Fine-tune top 15 layers
    for layer in base_model.layers[:-15]:
        layer.trainable = False
    for layer in base_model.layers[-15:]:
        layer.trainable = True

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", dtype='float32')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=CategoricalCrossentropy(label_smoothing=0.1),
        metrics=["accuracy"]
    )

    # Callbacks
    model_save_path = os.path.join(models_path, f"mobilenet_OCTDL_best_fold_{i}.keras")
    callbacks = [
        ModelCheckpoint(model_save_path, monitor='val_accuracy', save_best_only=True, verbose=1),
        EarlyStopping(monitor='val_accuracy', patience=8, verbose=1, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=4, verbose=1),
        CSVLogger(os.path.join(logs_path, f"mobilenet_OCTDL_fold_{i}.csv"), append=True)
    ]

    # Train
    model.fit(train_data, validation_data=val_data, epochs=epochs, class_weight=class_weight_dict, callbacks=callbacks)

    # Evaluate
    test_loss, test_accuracy = model.evaluate(test_data, verbose=1)

    # Predictions
    predictions = model.predict(test_data)
    predicted_classes = np.argmax(predictions, axis=1)
    true_labels = test_data.classes

    # Metrics
    report = classification_report(true_labels, predicted_classes, target_names=train_data.class_indices.keys(), output_dict=True)

    all_results.append({
        "Fold": i,
        "Test Accuracy": test_accuracy,
        "Test Loss": test_loss,
        "Precision": report["weighted avg"]["precision"],
        "Recall": report["weighted avg"]["recall"],
        "F1-Score": report["weighted avg"]["f1-score"]
    })

# Save results
results_df = pd.DataFrame(all_results)
results_excel_file = os.path.join(results_path, "mobilenet_OCTDL_5CV_results.xlsx")
results_df.to_excel(results_excel_file, index=False)
print(f"✅ All results saved to {results_excel_file}")



========== Training on Fold 1 ==========
Found 162 images belonging to 7 classes.
Found 65 images belonging to 7 classes.
Found 97 images belonging to 7 classes.


/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 568ms/step - accuracy: 0.1655 - loss: 2.4342
Epoch 1: val_accuracy improved from -inf to 0.06154, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/mobilenet/octdl/models/mobilenet_OCTDL_best_fold_1.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.1692 - loss: 2.4384 - val_accuracy: 0.0615 - val_loss: 3.1774 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.2600 - loss: 2.4681
Epoch 2: val_accuracy did not improve from 0.06154
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 318ms/step - accuracy: 0.2572 - loss: 2.4315 - val_accuracy: 0.0308 - val_loss: 3.2534 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - accuracy: 0.2333 - loss: 2.0997
Epoch 3: val_accuracy did not improve from 0.06154
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 352ms/step - accuracy: 0.2344 - loss: 2.0929 - val_accuracy: 0.0308 - val_loss: 2.8860 - learning_rate: 1.0000e-

/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 858ms/step - accuracy: 0.1846 - loss: 3.1759
Epoch 1: val_accuracy improved from -inf to 0.43077, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/mobilenet/octdl/models/mobilenet_OCTDL_best_fold_2.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.1874 - loss: 3.0955 - val_accuracy: 0.4308 - val_loss: 1.6107 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - accuracy: 0.2783 - loss: 2.7988
Epoch 2: val_accuracy did not improve from 0.43077
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 330ms/step - accuracy: 0.2782 - loss: 2.7665 - val_accuracy: 0.4308 - val_loss: 1.6386 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step - accuracy: 0.2262 - loss: 2.1179
Epoch 3: val_accuracy did not improve from 0.43077
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 393ms/step - accuracy: 0.2336 - loss: 2.1424 - val_accuracy: 0.4308 - val_loss: 1.6442 - learning_rate: 1.0000e-

...

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0707 - loss: 2.7032 
Epoch 1: val_accuracy improved from -inf to 0.23077, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/mobilenet/octdl/models/mobilenet_OCTDL_best_fold_4.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 0.0730 - loss: 2.6974 - val_accuracy: 0.2308 - val_loss: 1.8287 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step - accuracy: 0.1338 - loss: 2.4423
Epoch 2: val_accuracy improved from 0.23077 to 0.32308, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/mobilenet/octdl/models/mobilenet_OCTDL_best_fold_4.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 497ms/step - accuracy: 0.1395 - loss: 2.4572 - val_accuracy: 0.3231 - val_loss: 1.8931 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step - accuracy: 0.2683 - loss: 2.0698
Epoch 3: val_accura

/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4465 - loss: 1.9082
Epoch 1: val_accuracy improved from -inf to 0.44615, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/mobilenet/octdl/models/mobilenet_OCTDL_best_fold_5.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.4271 - loss: 1.9417 - val_accuracy: 0.4462 - val_loss: 1.6328 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.3081 - loss: 1.9977
Epoch 2: val_accuracy did not improve from 0.44615
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 352ms/step - accuracy: 0.3049 - loss: 1.9986 - val_accuracy: 0.3385 - val_loss: 1.8157 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step - accuracy: 0.2738 - loss: 2.1383
Epoch 3: val_accuracy did not improve from 0.44615
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 390ms/step - accuracy: 0.2755 - loss: 2.1199 - val_accuracy: 0.3692 - val_loss: 1.7815 - learning_rate: 1.0000e-04


/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

✅ All results saved to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/mobilenet/octdl/results/mobilenet_OCTDL_5CV_results.xlsx


In [4]:
#EfficientNet - OCTDL - 5CV
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Paths
dataset_base_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/dataset split/OCTDL_Splits"
logs_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientnet/octdl/training"
models_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientnet/octdl/models"
results_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientnet/octdl/results"

# Make sure directories exist
os.makedirs(logs_path, exist_ok=True)
os.makedirs(models_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)

# Hyperparameters
image_size = 224
batch_size = 32
epochs = 50
num_classes = 7

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    zoom_range=0.3,
    shear_range=0.2
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Results storage
all_results = []

for i in range(1, 6):
    print(f"\n========== Training on Fold {i} ==========")

    set_path = os.path.join(dataset_base_path, f"OCTDL_Set_{i}")
    train_dir, val_dir, test_dir = [os.path.join(set_path, x) for x in ["Train", "Validation", "Test"]]

    train_data = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical')
    val_data = val_test_datagen.flow_from_directory(val_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical')
    test_data = val_test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', shuffle=False)

    # Compute class weights
    labels = train_data.classes
    unique_classes = np.unique(labels)
    class_weights = compute_class_weight('balanced', classes=unique_classes, y=labels)
    class_weight_dict = {i: weight for i, weight in zip(unique_classes, class_weights)}

    # Load EfficientNetB0
    base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))
    
    # Fine-tune top 15 layers
    for layer in base_model.layers[:-15]:
        layer.trainable = False
    for layer in base_model.layers[-15:]:
        layer.trainable = True

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", dtype='float32')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=CategoricalCrossentropy(label_smoothing=0.1),
        metrics=["accuracy"]
    )

    # Callbacks
    model_save_path = os.path.join(models_path, f"efficientnet_OCTDL_best_fold_{i}.keras")
    callbacks = [
        ModelCheckpoint(model_save_path, monitor='val_accuracy', save_best_only=True, verbose=1),
        EarlyStopping(monitor='val_accuracy', patience=8, verbose=1, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=4, verbose=1),
        CSVLogger(os.path.join(logs_path, f"efficientnet_OCTDL_fold_{i}.csv"), append=True)
    ]

    # Train
    model.fit(train_data, validation_data=val_data, epochs=epochs, class_weight=class_weight_dict, callbacks=callbacks)

    # Evaluate
    test_loss, test_accuracy = model.evaluate(test_data, verbose=1)

    # Predict
    predictions = model.predict(test_data)
    predicted_classes = np.argmax(predictions, axis=1)
    true_labels = test_data.classes

    # Classification report
    report = classification_report(true_labels, predicted_classes, target_names=train_data.class_indices.keys(), output_dict=True)

    all_results.append({
        "Fold": i,
        "Test Accuracy": test_accuracy,
        "Test Loss": test_loss,
        "Precision": report["weighted avg"]["precision"],
        "Recall": report["weighted avg"]["recall"],
        "F1-Score": report["weighted avg"]["f1-score"]
    })

# Save results
results_df = pd.DataFrame(all_results)
results_excel_file = os.path.join(results_path, "efficientnet_OCTDL_5CV_results.xlsx")
results_df.to_excel(results_excel_file, index=False)
print(f"✅ All results saved to {results_excel_file}")



========== Training on Fold 1 ==========
Found 162 images belonging to 7 classes.
Found 65 images belonging to 7 classes.
Found 97 images belonging to 7 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1864 - loss: 1.9233
Epoch 1: val_accuracy improved from -inf to 0.50769, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientnet/octdl/models/efficientnet_OCTDL_best_fold_1.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 53s 6s/step - accuracy: 0.1774 - loss: 1.9387 - val_accuracy: 0.5077 - val_loss: 1.7939 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 790ms/step - accuracy: 0.1991 - loss: 1.6847
Epoch 2: val_accuracy did not improve from 0.50769
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.1980 - loss: 1.7377 - val_accuracy: 0.5077 - val_loss: 1.7697 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 672ms/step - accuracy: 0.1645 - loss: 2.2454
Epoch 3: val_accuracy did not improve from 0.50769
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 956ms/step - accuracy: 0.1666 - loss: 2.1917 - val_accuracy: 0.5077 - val_loss: 1.7541 - learning_rate: 1.0000e-

/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1143 - loss: 1.9485   
Epoch 1: val_accuracy improved from -inf to 0.01538, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientnet/octdl/models/efficientnet_OCTDL_best_fold_2.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 129s 21s/step - accuracy: 0.1156 - loss: 1.9612 - val_accuracy: 0.0154 - val_loss: 1.9872 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 875ms/step - accuracy: 0.1574 - loss: 2.1741
Epoch 2: val_accuracy did not improve from 0.01538
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.1552 - loss: 2.1558 - val_accuracy: 0.0154 - val_loss: 1.9800 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 511ms/step - accuracy: 0.1087 - loss: 2.0795
Epoch 3: val_accuracy did not improve from 0.01538
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 920ms/step - accuracy: 0.1117 - loss: 2.0737 - val_accuracy: 0.0154 - val_loss: 1.9653 - learning_rate: 1.

...

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1549 - loss: 2.1525   
Epoch 1: val_accuracy improved from -inf to 0.06154, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientnet/octdl/models/efficientnet_OCTDL_best_fold_4.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.1496 - loss: 2.1461 - val_accuracy: 0.0615 - val_loss: 1.8448 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 529ms/step - accuracy: 0.1442 - loss: 2.1569
Epoch 2: val_accuracy did not improve from 0.06154
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 823ms/step - accuracy: 0.1431 - loss: 2.1426 - val_accuracy: 0.0615 - val_loss: 1.8500 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 592ms/step - accuracy: 0.2125 - loss: 2.0268
Epoch 3: val_accuracy did not improve from 0.06154
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 854ms/step - accuracy: 0.2150 - loss: 2.0199 - val_accuracy: 0.0615 - val_loss: 1.8679 - learning_rate: 1.

/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.1473 - loss: 1.9765  
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientnet/octdl/models/efficientnet_OCTDL_best_fold_5.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 399s 29s/step - accuracy: 0.1502 - loss: 1.9702 - val_accuracy: 0.0000e+00 - val_loss: 2.0618 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1635 - loss: 1.8980  
Epoch 2: val_accuracy did not improve from 0.00000
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.1658 - loss: 1.9070 - val_accuracy: 0.0000e+00 - val_loss: 2.0456 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 911ms/step - accuracy: 0.1990 - loss: 2.3803
Epoch 3: val_accuracy improved from 0.00000 to 0.03077, saving model to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientne

/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

✅ All results saved to /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientnet/octdl/results/efficientnet_OCTDL_5CV_results.xlsx
